In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
path = "/content/drive/MyDrive/fNIRS_Deeplearning_Test"
os.chdir(path)
os.listdir(path)


['30subjects-20220706T144134Z-001.zip',
 'dataset',
 'wandb',
 'model_run_exis_sweep.ipynb',
 'model_run_exis_sweep_20220903.ipynb',
 'src_before20230324',
 'Untitled0.ipynb',
 'src_before20230417',
 'model_run_exis_sweep_20230417.ipynb',
 'project.csv',
 'src_before20230509',
 'model_run_exis_sweep_20230323.ipynb',
 'model_run_exis_sweep_20230509.ipynb',
 'other_model__20230509.ipynb',
 'Untitled1.ipynb',
 'src_before20231225',
 'src',
 'Demo.ipynb',
 'vistualization (1).ipynb',
 'finish_time.txt',
 'download_wandb_sweep.ipynb',
 'Demo_2.ipynb',
 'Demo_SVM_KNN.ipynb']

In [3]:
import sys

path_to_this_work = '/content/drive/MyDrive/fNIRS_Deeplearning_Test'
path_to_src = path_to_this_work + '/src'

sys.path.insert(0, path_to_src)
os.environ['PYTHONPATH'] += (":"+path_to_src)
from BCI_model_class import SCCNet, SCCNet_25s, ShallowConvNet, ShallowConvNet2, cus_EEGNet
import Dataset_training_schema as dts
import train_model as tm

In [4]:
import numpy as np
import math

In [5]:
!pip install wandb

In [6]:
import wandb

In [7]:
wandb.login(key="550fa288f18a4938b6519ecd67d11309cfd9d51e")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [9]:
def average_pooling_by_interval(matrix, m):
  # 輸入 要進行處理的矩陣matrix 以及 要抽取的特徵點數量
  # matrix --> fNIRS 資料， (channel * time point)
  # m --> 以時間區段作為區分，一個channel要抽取 m 特徵點
  # 輸出
  # 處理後的資料 (channel * m)
  # 進行特徵抽取之後的資料，輸出的維度為

  n_samples, n_channels, time_points = matrix.shape

  # 區間理論大小
  interval_size = time_points / m

  # 初始化結果矩陣
  pooled_matrix = np.zeros((n_samples, n_channels, m))

  # 對每個樣本進行處理
  for i in range(n_samples):
      start = 0
      for j in range(m):

          # 計算裡親間終點
          end = start + interval_size

          # 擷取區間
          reshaped_matrix = matrix[i][:, round(start):round(end)].reshape(n_channels, -1)
          averaged_value = np.mean(reshaped_matrix, axis=1)
          pooled_matrix[i][:, j] = averaged_value

          # 更新區間起始
          start = end

  return pooled_matrix

In [10]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier


def runClassify(method_model, data, label, testData=0, testLabel=0):


  # if method=="SVM":
  #   # 創建 SVM 的模型，使用線性函數
  #   method_model = SVC(C=1, kernel='linear', probability=True)

  # elif method == "KNN":
  #   # 創建KNN分類器
  #   method_model = KNeighborsClassifier(n_neighbors=5)


  # 一對一的多類別分類
  model = OneVsOneClassifier(method_model)



  if isinstance(testData, int) and testData == 0 :
  # SD

    predictions = []  # 預測結果
    true_labels = []  # 真實標籤

    # Leave-One-Out CV
    loo = LeaveOneOut()

    # 依序進行 Leave-One-Out 驗證
    for train_index, test_index in loo.split(data):
        # 分割训练集和测试集
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = label[train_index], label[test_index]

        # 數據標準化
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # 模型訓練
        model.fit(X_train, y_train)

        # 預測
        prediction = model.predict(X_test)
        # print(X_test.shape)

        predictions.append(prediction[0]) # 預測結果
        true_labels.append(y_test[0]) # 真實結果

    # 分類精準度
    accuracy = accuracy_score(true_labels, predictions)
  else:
  # SI

    # 數據標準化
    scaler = StandardScaler()
    train_data = scaler.fit_transform(data)
    test_data = scaler.fit_transform(testData)

    #模型訓練
    model.fit(train_data, label)
    prediction = model.predict(test_data)

    # 分類精準度
    accuracy = accuracy_score(testLabel, prediction)

  return accuracy*100


In [11]:
def feature_extrat(fnirs_data, total_ch_feature_point, filter_ch_feature_point):


  feature_HbO = average_pooling_by_interval(fnirs_data.HbO,total_ch_feature_point)
  feature_HbR = average_pooling_by_interval(fnirs_data.HbR,total_ch_feature_point)
  my_feature = np.concatenate((feature_HbO[:,np.newaxis,:,:], feature_HbR[:,np.newaxis,:,:]), axis=1)

  sample_num = my_feature.shape[0]
  d = my_feature.shape[1]*my_feature.shape[2]
  data = my_feature[:,:,:,0:filter_ch_feature_point].reshape(sample_num, d*filter_ch_feature_point)
  label = np.argmax(fnirs_data.labels, axis=1)
  return data, label

In [16]:
def main(dataset,duration,total_ch_feature_point,ch_feature_num,sub,scheme,method):
# def main():

  # wandb.init()
  # # Config is a variable that holds and saves hyperparameters and inputs
  # config = wandb.config
  # total_ch_feature_point=5
  # ch_feature_num = 3
  # dataset = config.dataset
  # duration = config.duration
  # sub = int(config.sub)
  # scheme = config.scheme
  # method = config.method

  if (int(sub)>30 and dataset=='preprocess') or (int(sub)>29 and dataset in ['MI','MA']):
    return 0

  if method=="SVM":
    # 創建 SVM 的模型，使用線性函數
    method_model = SVC(C=1, kernel='linear', probability=True)

  elif method == "KNN":
    # 創建KNN分類器
    method_model = KNeighborsClassifier(n_neighbors=5)

  path_dataset='./dataset/'+ dataset + '_' + duration + '/'
  s1=path_dataset+'S'
  s2='.mat'
  sub_range = 31 if dataset=='preprocess' else 30

  if scheme=="SI":

    # 檔案讀取
    test, train = dts.leave_subject_out(s1,s2, data_range=range(1,sub_range), test_id=int(sub))

    # 特徵提取
    train_data, train_label = feature_extrat(train, total_ch_feature_point=total_ch_feature_point, filter_ch_feature_point=ch_feature_num)
    test_data, test_label = feature_extrat(test, total_ch_feature_point=total_ch_feature_point, filter_ch_feature_point=ch_feature_num)


    acc =  runClassify(method_model, train_data, train_label, testData=test_data, testLabel=test_label)


    # wandb.log({"Accuracy": acc})
    # wandb.finish()

    print('ACC: ',acc)
    return 0



  elif scheme=="SD":

    # 檔案讀取
    data = dts.read_one_file(s1,s2,sub)

    # 特徵提取
    data, label = feature_extrat(data, total_ch_feature_point=5, filter_ch_feature_point=ch_feature_num)

    acc = runClassify(method_model,data,label)


    # wandb.log({"Accuracy": acc})
    # wandb.finish()
    print('ACC: ',acc)
    return 0


In [ ]:
projectName = 'SVM_KNN_eval'
sweepID = 'zfhhdori'
sweep_agent = 'cphnycu/' + projectName + '/' + sweepID
wandb.agent(sweep_agent, main)

In [17]:
dataset = 'preprocess'
duration = '25s'
ch_feature_num = 3
sub = 1
scheme = "SD"
method = "KNN"

In [18]:
main(dataset = 'MA',duration = '25s',total_ch_feature_point=5, ch_feature_num = 3,sub = 1,scheme = "SI",method = "SVM")

leave subject out
train id:2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, test id:  1
training data dimension:  (1680, 36, 251)
training label dimension:  (1680, 2)
testing data dimension:  (60, 36, 251)
testing label dimension:  (60, 2)
ACC:  76.66666666666667


0